# 🌞 Example Weather App
> Using `prodb`


* Starting with a `.csv` of locations, we wish to call the [MetaWeather](https://www.metaweather.com/api/) API to pull in the weather forecast.  
* The app needs the ability to input new locations by the user

 🌍 Core functions will be converted into the streamlit app `weather_app.py`

In [ ]:
#hide
from nbdev.showdoc import *
import pandas as pd
%load_ext autoreload
%autoreload 2

## 1. Requests to `metaweather` open weather API

In [ ]:
import requests
import json
import arrow

In [ ]:
def get_json_from_query(location):
    """Search for a city and return metadata from API"""
    url = f"https://www.metaweather.com/api/location/search/?query={location}"
    r = requests.get(url).json()
    return r[0]

l = get_json_from_query('London')
l

{'title': 'London',
 'location_type': 'City',
 'woeid': 44418,
 'latt_long': '51.506321,-0.12714'}

Check `woeid`(Where On Earth ID) is correct for London

In [ ]:
assert l['woeid'] == 44418

Get weather data for the London `woeid`

In [ ]:
utc = arrow.utcnow().format('YYYY/MM/DD')
utc

'2021/10/03'

In [ ]:
def get_current_weather(woeid, date='today'):
    """
    inputs:
    woeid int: i.e. 44418 for london
     date str: e.e '2021/05/05'
    """
    if date == 'today':
        date = arrow.utcnow().format('YYYY/MM/DD')
    url = f"https://www.metaweather.com/api/location/{woeid}/"
    res = requests.get(url).json()
    return res

res = get_current_weather(l['woeid'])
res['time']

'2021-10-03T16:01:15.248959+01:00'

There are a number of metrological properties available to us from the API. 

The ones we are most interested in are:
* `the_temp` 🌡️ current temperature
* `weather_state_name` ⛅ current sky condition
* `max_temp` 🥵 daily max temp
* `min_temp` 🥶 daily min temp


In [ ]:
loc = get_json_from_query(query)
res = get_current_weather(loc['woeid'])
df =  pd.DataFrame.from_records(res[:2])
df['readable_time'] = df.created.apply(lambda x: arrow.get(x).humanize())
df

,id,weather_state_name,weather_state_abbr,wind_direction_compass,created,applicable_date,min_temp,max_temp,the_temp,wind_speed,wind_direction,air_pressure,humidity,visibility,predictability,readable_time
0,5495230276042752,Light Rain,lr,SW,2021-10-03T12:59:03.049755Z,2021-10-03,10.645,16.015,14.400,8.614009,227.333465,997.0,75,11.977551,75,an hour ago
1,5479289236488192,Light Rain,lr,SW,2021-10-03T09:59:02.071081Z,2021-10-03,10.565,16.015,14.515,9.690802,227.666907,997.5,71,13.347985,75,4 hours ago


In [ ]:
def df_from_loc(query, 
                date='today',
                n_hist=1,
                keep_cols='location the_temp readable_time created weather_state_name min_temp max_temp'.split(' ')):
    loc = get_json_from_query(query)
    res = get_current_weather(loc['woeid'])
    df =  pd.DataFrame.from_records(res[:n_hist])
    df['location'] = query
    df['readable_time'] = df.created.apply(lambda x: arrow.get(x).humanize())
    return df[keep_cols]


Example output showing location and current weather situation

In [ ]:
df = df_from_loc('London')
df

,location,the_temp,readable_time,created,weather_state_name,min_temp,max_temp
0,London,14.4,an hour ago,2021-10-03T12:59:03.049755Z,Light Rain,10.645,16.015


In [ ]:
df = df_from_loc('Auckland')
df

,location,the_temp,readable_time,created,weather_state_name,min_temp,max_temp
0,Auckland,16.23,4 hours ago,2021-10-03T10:37:05.045379Z,Showers,8.65,16.685


In [ ]:
loc = get_json_from_query('Auckland')
res = get_current_weather(l['woeid'])
res[0]

{'id': 5495230276042752,
 'weather_state_name': 'Light Rain',
 'weather_state_abbr': 'lr',
 'wind_direction_compass': 'SW',
 'created': '2021-10-03T12:59:03.049755Z',
 'applicable_date': '2021-10-03',
 'min_temp': 10.645,
 'max_temp': 16.015,
 'the_temp': 14.4,
 'wind_speed': 8.614009459582704,
 'wind_direction': 227.33346500159422,
 'air_pressure': 997.0,
 'humidity': 75,
 'visibility': 11.97755110156685,
 'predictability': 75}